In [55]:
import numpy as np

import matplotlib.pyplot as plt

from random import random
from copy import deepcopy

In [56]:
X = np.reshape([random()*100 for i in range(400)],(100,4))

In [57]:
col_max = np.amax(X,axis=0)
col_min = np.amin(X,axis=0)
centroid = (col_max + col_min)/2

In [58]:
Z = (X-centroid)**2

In [59]:
alpha = (1.0 / 100) * np.ones(100)

In [60]:
ZT = Z.T
h = np.matmul(alpha,Z)
h

array([973.9179621 , 943.87804881, 779.86768031, 698.43487408])

In [61]:
def isclose(a, b, rel_tol=1e-02, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [40]:
isclose(1,1.02)


False

In [68]:
def calc_s(h,alpha_old,Z,u,i):
  ans=0
  for j in range(4):
            s_i_j = h[j] - alpha_old[i]*Z[i][j]
            ans += (1/(u + s_i_j/Z[i][j]))
  return ans

In [69]:
def solution(Z,alpha,h):
    i = 0
    updates = 0
    converged = False
    alpha_old = np.zeros(100)
    old = 0
    start=0

    
    while i < 100:
        if converged:
            return h
        alpha_old[i] = alpha[i]
        f_x = 0
        u = random()
        f_x=calc_s(h,alpha_old,Z,u,i)
        
        if f_x == 1:
            alpha[i] = u
        
        if f_x < 1:

            alpha[i] = 0

        if f_x > 1:
            u+=0.00001*u
            while(not isclose(f_x,1)):
                f_x = calc_s(h,alpha_old,Z,u,i)
                if(f_x>1):
                    u+=0.0001*u
                else:
                    u-=0.0001*u 
            alpha[i]=u    
            
                
        h = h + (alpha[i] - alpha_old[i])*Z[i]
        i += 1
        if i>=99 :
            i = 0
            updates += 1
            if old == 0:
                stored_alpha_old = deepcopy(alpha_old)
                old = 1
        if updates == 4:
            if np.all(alpha - stored_alpha_old) < .00001:
                converged = True
            else:
                updates = 0
                stored_alpha_old = deepcopy(alpha)
    

In [70]:
h = solution(Z,alpha,h)
h

array([ 7806.71740908,  5249.13344947, 10106.82920625,  7713.43229188])

In [71]:
v = 1/h
v

array([1.28094812e-04, 1.90507635e-04, 9.89429998e-05, 1.29643972e-04])

In [ ]:
def coordinatedescent(S,Z,i,var_x):
  sum=0
  for j in range(4):
    sum+=1/var_x[i,j]+(S[i,j]/Z[i,j])
  if(sum==1):
    return var_x[i] 


In [ ]:
def getS(h,alpha,Z,n,d,i):
  S = np.reshape([random()*100 for io in range(400)],(100,4))
  for j in range(4):
    S[i,j]= h[j]-alpha[j]*Z[i,j]
  return S


In [ ]:
def meepcalgo(n):
  index= np.random.randint(n)
  converged = False
  # alpha_i=X[index,:]
  while(index<100):
    if converged:
      break
    alpha_old = alpha[index]
    S = getS(h,alpha_old,Z,n,4,index)
    var_x=np.reshape([random() for i in range(100)],(100,4))
    f_val=coordinatedescent(S,Z,index)
    index+=1
    if(index==100):
      index=0